In [ ]:
%pip install -U pip setuptools wheel
%pip install -U spacy


In [ ]:
!python -m spacy download en_core_web_sm


In [13]:
import spacy
import json

nlp = spacy.load("en_core_web_sm")


In [39]:
!node run.js 15


--- Arbroath Abbey
--- Abano Terme
--- Aeclanum
--- Aegadian Islands
--- Ajaigarh
--- Ajanta Caves
--- Ajmer
--- Afyonkarahisar
--- Abdera, Spain
--- Aberavon (UK Parliament constituency)
--- Abercarn
--- Aberdeen
error creating shortDescription
--- Actium
--- Abadeh
--- Abae
--- Abakan
--- Aswan
--- Arapaoa Island
--- Ames, Iowa
--- Abensberg
--- Andes
--- Anazarbus
--- Anadyr (river)
--- Ambracia
--- Abydos, Egypt
--- Azincourt
--- Achill Island
--- Abadan, Iran
--- Adobe Inc.
error creating shortDescription
--- Anguilla
--- Ashmore and Cartier Islands
--- AFC Ajax
--- Area 51
--- Albion, Michigan
--- Alicante
--- Austin, Texas
--- Crandall University
--- Acadia University
--- Annapolis Valley
--- Alberta
--- Ankara
--- Aberdeenshire
Artikel wurden als JSON gespeichert.


In [14]:
with open('parsed_articles.json') as file:
    parsed_articles = json.load(file)


In [15]:
def clean_str(string: str):
  return string.replace(
                '\n', ' ').replace('  ', ' ').strip()


In [16]:
articles = []
for article in parsed_articles:
    doc = nlp(article['history'])
    sentences = list(doc.sents)
    sentences_with_dates = []

    for ent in doc.ents:
        if ent.label_ == "DATE":
            current_sentence = ent.sent
            current_index = sentences.index(current_sentence)
            current_sentence_text = clean_str(current_sentence.text)

            # Hinzufügen des vorherigen Satzes, wenn vorhanden und noch nicht in der Liste
            # if current_index > 0 and sentences[current_index - 1].text not in sentences_with_dates:
                # sentences_with_dates.append(sentences[current_index - 1].text)

            # Hinzufügen des aktuellen Satzes, wenn noch nicht in der Liste
            if current_sentence_text not in sentences_with_dates:
                sentences_with_dates.append(current_sentence_text)

            # Hinzufügen des nachfolgenden Satzes, wenn vorhanden und noch nicht in der Liste
            # if current_index < len(sentences) - 1 and sentences[current_index + 1].text not in sentences_with_dates:
                # sentences_with_dates.append(sentences[current_index + 1].text)
        
    article['sentencesWithDates'] = ' '.join(sentences_with_dates)
    articles.append(article)


In [17]:
len(articles)


42

In [18]:
import re


def remove_first_parentheses_in_first_sentence(text):
    sentences = re.split(r'(?<=[.!?])\s+', text, maxsplit=1)
    first_sentence = sentences[0]
    remaining_text = sentences[1] if len(sentences) > 1 else ''
    cleaned_first_sentence = re.sub(r'\(.*?\)', '', first_sentence, count=1)
    return cleaned_first_sentence + ' ' + remaining_text


In [19]:
cleaned_articles = []
for article in articles:
    if 'history' in article:
        article['history'] = article['history']
    if 'title' in article:
        article['title'] = article['title']
    if 'shortDescription' in article:
        article['shortDescription'] = article['shortDescription']
    if 'description' in article:
        article['description'] = article['description']

    cleaned_articles.append(article)


In [20]:
with open('articles_with_dates.json', 'w') as file:
    json.dump(cleaned_articles, file, ensure_ascii=False,)


In [ ]:
for article in articles:
    print(
        article['title'], len(article['history']), len(article['sentencesWithDates']))
    print('------------------')


In [ ]:
%pip install openai python-dotenv


In [8]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
client = OpenAI(
  api_key=os.getenv("OPENAI_API_KEY")
)


In [11]:
def get_messages(text):
  return [
    {
      "role": 'system',
      "content": 'Your task is to extract dates and related event information from the provided text.',
    },
    {
      "role": 'user',
      "content": f'Extract dates from this text: {text}',
    },
  ]
  

In [5]:
tools = [
    {"type": "function",
     "function": {
         "name": 'extract_dates',
         "description": 'Extracts dates and related information from text.',
         "parameters": {
             "type": 'array',
             "description": 'Extracted dates',
             "items": {
                        "type": 'object',
                        "properties": {
                            "shortTitle": {
                                "type": 'string',
                                "description": "Title of the event. NOT LONGER THAN 3 WORDS"
                            },
                            "description": {
                                "type": 'string',
                                "description": 'Description of the event. NOT LONGER THAN 3 SENTENCES',
                            },
                            "dates": {
                                "type": 'array',
                                "description": 'Dates of the event. One date if it is a point in time, two if it is a period of time',
                                "items": {
                                    "type": 'object',
                                    "properties": {
                                        "date": {
                                            "type": 'string',
                                            "description": 'Date of the event. ALWAYS IN YYYY-MM-DD FORMAT',
                                        },
                                        "scale": {
                                            "type": 'string',
                                            "description": 'Scale of the event',
                                            "enum": [
                                                "day",
                                                "month",
                                                "year",
                                                "decade",
                                                "century",
                                                "millennium"
                                            ]
                                        },
                                    }
                                }
                            }
                        }
             },
         },
     }, }
]


In [21]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=get_messages(cleaned_articles[0]['sentencesWithDates']),
    tools=tools,
    tool_choice="extract_dates"
)
response


BadRequestError: Error code: 400 - {'error': {'message': "'$.tool_choice' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}